In [10]:
#This notebook is for testing the download of PIPS data using the web API.
#It uses urllib3 and BeautifulSoup4
%matplotlib notebook
import os
from glob import glob
import pandas as pd
import xarray as xr
import numpy as np
from datetime import datetime

In [14]:
# From https://stackoverflow.com/questions/6618515/sorting-list-based-on-values-from-another-list
def sortby(X, Y):
    """Sorts list X by values in list Y"""
    return [x for _, x in sorted(zip(Y, X), key=lambda pair: pair[0])]

In [18]:
netcdf_dir = '/Users/dawson29/sshfs_mounts/depot/data/Projects/TriPIPS/netcdf_web/'
onesec_files = glob(netcdf_dir + 'onesec_*')
# print(*[os.path.basename(f) for f in onesec_files], sep='\n')
timestampfmt = '%Y%m%d%H%M%S'
timestamplist = [datetime.strptime(os.path.basename(f)[7:21], timestampfmt) for f in onesec_files]
indices = sorted(range(len(timestamplist)), key=lambda k: timestamplist[k])
timestamplist_sorted = [timestamplist[i] for i in indices]
onesec_files_sorted = [onesec_files[i] for i in indices]

In [ ]:
for file in onesec_files_sorted[1:]:
    

In [ ]:
#%%time
#for i in range(numiter):
while True:
    ax.clear()
    ax_vd.clear()
    ax_t_td.clear()
    ax_windspd.clear()
    ax_winddir.clear()
    ax_pressure.clear()
    onesec_new_df = scrape_tripips_onesec_data(url_onesec, numrecords=numrecords_append_onesec)
    onesec_new_df['Dewpoint'] = thermo.calTdfromRH(onesec_new_df['Pressure'] * 100., onesec_new_df['SlowTemp'] + 273.15, 
                                     onesec_new_df['RH'] / 100.) - 273.15
    # Append new data onto onesec_df
    onesec_df = onesec_df.append(onesec_new_df)
    # Drop duplicate timestamps
    onesec_df = onesec_df[~onesec_df.index.duplicated(keep='first')]
    # Drop records older than desired interval
    last_timestamp_onesec = onesec_df.index[-1]
    oldest_timestamp_onesec = last_timestamp_onesec-keep_data_for_ts
    onesec_df = onesec_df.loc[oldest_timestamp_onesec:]
    # Dump onesec dataframe to netCDF file (via xarray)
    netcdf_filename = 'onesec_{}_{}.nc'.format(oldest_timestamp_onesec.strftime('%Y%m%d%H%M%S'), 
                                               last_timestamp_onesec.strftime('%Y%m%d%H%M%S' ))
    netcdf_path = os.path.join(netcdf_output_dir, netcdf_filename)
    onesec_df.to_xarray().to_netcdf(netcdf_path)
    plottimes_onesec = [onesec_df.index.to_pydatetime()]
    # Temperature and Dewpoint
    fields_to_plot_onesec = [onesec_df['SlowTemp'].values, onesec_df['Dewpoint'].values]
    field_parameters_onesec = [pm.temp_params, pm.dewpoint_params]
    ax_t_td = pm.plotmeteogram(ax_t_td, plottimes_onesec, fields_to_plot_onesec, field_parameters_onesec)
    temp_dewp_ax_params['axeslimits'][0] = (plottimes_onesec[0][0], plottimes_onesec[0][-1]) 
    ax_t_td, = pm.set_meteogram_axes([ax_t_td], [temp_dewp_ax_params])
    # Wind speed and direction
    ax_windspd = pm.plotmeteogram(ax_windspd, plottimes_onesec, [onesec_df['WS_ms'].values], [pm.windspeed_params])
    ax_winddir = pm.plotmeteogram(ax_winddir, plottimes_onesec, [onesec_df['WindDir'].values], [pm.winddir_params])
    windspd_ax_params['axeslimits'][0] = (plottimes_onesec[0][0], plottimes_onesec[0][-1])
    winddir_ax_params['axeslimits'][0] = (plottimes_onesec[0][0], plottimes_onesec[0][-1])
    ax_windspd, ax_winddir = pm.set_meteogram_axes([ax_windspd, ax_winddir], [windspd_ax_params, winddir_ax_params])
    # Pressure
    pmin = np.nanmin(onesec_df['Pressure'].values)
    pmax = np.nanmax(onesec_df['Pressure'].values)
    pressure_ax_params['axeslimits'] = [None, [pmin - 2.5, pmax + 2.5]]
    fields_to_plot_press = [onesec_df['Pressure'].values]
    field_parameters_press = [pm.pressure_params]
    ax_pressure = pm.plotmeteogram(ax_pressure, plottimes_onesec, fields_to_plot_press, field_parameters_press)
    ax_pressure, = pm.set_meteogram_axes([ax_pressure], [pressure_ax_params])
    

    # DSD plots
    telegram_new_df, spectrum_new_da = scrape_tripips_tensec_data(url_tensec, numrecords=numrecords_append_tensec)
    ND_new_da = calc_ND_da(spectrum_new_da)
    # Append new data onto the data array
    ND_da = xr.concat([ND_da, ND_new_da], dim='time')
    # Drop duplicate timestamps
    ND_da = ND_da.groupby('time').first()
    # onesec_df = onesec_df[~onesec_df.index.duplicated(keep='first')]
    # Drop records older than desired interval
    last_timestamp = ND_da['time'].to_index()[-1]
    spectrum = spectrum_new_da.loc[last_timestamp]
    # print(last_timestamp)
    oldest_timestamp = last_timestamp-keep_data_for_ts
    ND_da = ND_da.loc[oldest_timestamp:]
    # Dump ND_da to netcdf file
    
    netcdf_filename = 'ND_{}_{}.nc'.format(oldest_timestamp.strftime('%Y%m%d%H%M%S'), 
                                       last_timestamp.strftime('%Y%m%d%H%M%S' ))
    netcdf_path = os.path.join(netcdf_output_dir, netcdf_filename)
    ND_da.to_dataset(name='ND').to_netcdf(netcdf_path)
                                
    plottimes = [ND_da['time'].to_index().to_pydatetime()]
    ND_arr = ND_da.values.T
    logND_arr = np.ma.log10(ND_arr)
    fields_to_plot = [logND_arr]
    field_parameters = [log_ND_params]
    ax = pm.plotmeteogram(ax, plottimes, fields_to_plot, field_parameters, 
                          yvals=[min_diameter] * len(fields_to_plot))
    ax, = pm.set_meteogram_axes([ax], [log_ND_ax_params])
    ax_vd.set_title('Fall speed vs. diameter for time {0}'.format(last_timestamp.strftime(tm.timefmt2)))
    countsplot = np.ma.masked_where(spectrum.values <= 0, spectrum)
    C = ax_vd.pcolor(min_diameter, min_fall_bins, countsplot, vmin=1, vmax=50, edgecolors='w')
    divider = make_axes_locatable(ax_vd)
    cax = divider.append_axes("right", size="5%")
    cb = fig_vd.colorbar(C, cax=cax, orientation='vertical')
    ax_vd.set_xlim(0.0, 10.0)
    ax_vd.xaxis.set_major_locator(ticker.MultipleLocator(2.0))
    ax_vd.set_xlabel('diameter (mm)')
    ax_vd.set_ylim(0.0, 10.0)
    ax_vd.yaxis.set_major_locator(ticker.MultipleLocator(1.0))
    ax_vd.set_ylabel('fall speed (m/s)')
    
    display.display(fig)
    display.display(fig_vd)
    display.display(fig_t_td)
    display.display(fig_wind)
    display.display(fig_pressure)
    display.clear_output(wait=True)
    # fig.canvas.draw()
    fig.savefig(os.path.join(image_output_dir, 'logND_current.png'), dpi=300)
    fig_vd.savefig(os.path.join(image_output_dir, 'VD_current.png'), dpi=300)
    fig_t_td.savefig(os.path.join(image_output_dir, 'T_Td_current.png'), dpi=300)
    fig_wind.savefig(os.path.join(image_output_dir, 'wind_current.png'), dpi=300)
    fig_pressure.savefig(os.path.join(image_output_dir, 'pressure.png'), dpi=300)
    
    # plt.pause(0.01)
    # Sleep for the desired interval. This may not be perfectly accurate
    time.sleep(plot_update_interval)